<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterer-preproc

Saving kaggle.json to kaggle.json
 99% 203M/206M [00:03<00:00, 73.6MB/s]
100% 206M/206M [00:03<00:00, 55.9MB/s]


In [ ]:
!unzip /content/betterer-preproc.zip

In [2]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

 67% 23.0M/34.1M [00:00<00:00, 48.9MB/s]
100% 34.1M/34.1M [00:00<00:00, 63.7MB/s]


In [ ]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

# PRE PROCESS
.JPG TO .NPY

In [4]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/content/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [5]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, f"{input_folder}{i}.npy")
            np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [6]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


5121

In [7]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 52 2560 1792


In [8]:
import imgaug.augmenters as iaa

# Define the augmenter
augmenter = iaa.Sequential([
    iaa.Multiply((0.8, 1.2)),
    iaa.LinearContrast((0.8, 1.2)),
])

augmented_images = []

for path in moderateDem:
    path = os.path.join("/kaggle/working/train", path)
    image = np.load(path)

    for _ in range(20):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [9]:
len(augmented_images)

1040

In [10]:
import cv2
import os


output_directory = "/kaggle/working/train"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
    np.save(output_path, image)


print("Augmented images saved successfully.")


Augmented images saved successfully.


In [11]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

6161

In [12]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 1092 2560 1792


In [44]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [14]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [15]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[1 3 0 ... 3 3 1]
[1 0 2 ... 3 3 2]


In [17]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(4928, 208, 176, 3) (1233, 208, 176, 3) (4928,) (1233,)


In [18]:
print(test_labels)
test_labels_encoded

[1 0 2 ... 3 3 2]


array([1, 0, 2, ..., 3, 3, 2])

In [46]:
print(x.index, x.values, x.array)

Index([2, 3, 0, 1], dtype='int64', name='Class') [640 448 179  12] <PandasArray>
[640, 448, 179, 12]
Length: 4, dtype: int64


In [48]:
s = x / len(files)
class_weights = {i:j for i, j in zip(s.index, s.values)}

class_weights# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


{2: 0.5003909304143862,
 3: 0.3502736512900704,
 0: 0.13995308835027365,
 1: 0.009382329945269743}

In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN model
from tensorflow.keras import regularizers, layers, models

l2_reg = regularizers.l2(0.01)

from tensorflow.keras import backend as K


model = models.Sequential([
    layers.Input(shape=train_images[0].shape),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  # Dropout with 20% probability
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.Flatten(),
    layers.Dropout(0.2),  # Dropout with 20% probability
    layers.Dense(64, activation='relu', kernel_regularizer=l2_reg),
    layers.Dense(4)
])



class WeightedCategoricalCrossentropy(tf.keras.losses.CategoricalCrossentropy):
  def __init__(self, num_classes, weights, from_logits=True, **kwargs):
    super(WeightedCategoricalCrossentropy, self).__init__(from_logits=from_logits, **kwargs)
    self.num_classes = num_classes
    self.weights = K.variable(weights, dtype=tf.float32)

  def call(self, y_true, y_pred):
    y_true = K.cast(y_true, tf.float32)  # Ensure float32 type
    weights = K.tile(self.weights, [K.shape(y_true)[0], 1])  # Tile weights for each sample
    return super().call(y_true * weights, y_pred)

class_weights = np.array([class_weights[i] for i in range(len(class_weights))])
# class_weights = compute_class_weights(y_true, normalize=True)

# Compile the model with weighted loss
model.compile(optimizer='adam',
              loss=WeightedCategoricalCrossentropy(num_classes=4, weights=class_weights),
              metrics=['accuracy'])



# Train the model
history = model.fit(train_images, train_labels_encoded, epochs=15, validation_data=(test_images, test_labels_encoded))

Epoch 1/15


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-50-8a2c13384052>", line 34, in call  *
        weights = K.tile(self.weights, [K.shape(y_true)[0], 1])  # Tile weights for each sample
    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend.py", line 3901, in tile
        return tf.tile(x, n)

    ValueError: Shape must be rank 1 but is rank 2 for '{{node categorical_crossentropy/Tile}} = Tile[T=DT_FLOAT, Tmultiples=DT_INT32](categorical_crossentropy/Tile/ReadVariableOp, categorical_crossentropy/Tile/multiples)' with input shapes: [4], [2].


In [21]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)

y_prob = tf.nn.softmax(y_prob).numpy()

y_true = np.eye(4)[test_labels_encoded]

# Compute ROC AUC for each class
roc_auc = roc_auc_score(y_true, y_prob, average='macro')

print("ROC AUC:", roc_auc)


39/39 [==============================] - 1s 21ms/step
ROC AUC: 0.9613698484175601


In [ ]:
# Save the model
model.save("my_cnn_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
import os
import numpy as np
directory = '/content/Alzheimer_s Dataset/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1279

In [23]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [24]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/Alzheimer_s Dataset/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [25]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [26]:
labels = df["Class"]
labels = label_encoder.transform(labels)
# Verify the encoded labels
print(labels.shape)

(1279,)


In [31]:
y_prob = model.predict(images)

# y_prob = tf.nn.softmax(y_prob).numpy()
# Get the class with the highest probability for each prediction
y_prob = np.argmax(y_prob, axis=1)  # Argmax gives index of max value
for i in range(len(labels)):
  print(labels[i], y_prob[i])

40/40 [==============================] - 1s 19ms/step
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 0
0 2
0 0
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 1
0 0
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 2
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 0
1 0
1 2
1 0
1 1
1 1
1 0
1 1
1 1
1 0
1 1
1 0
1 0
2 1
2 0
2 1
2 1
2 1
2 1
2 0
2 0
2 1
2 0
2 1
2 0
2 0
2 0
2 1
2 1
2 0
2 1
2 0
2 1
2 1
2 1
2 1
2 0
2 1
2 1
2 0
2 1
2 1
2 1
2 1
2 1
2 2
2 0
2 1
2 0
2 0
2 1
2 2
2 1
2 0
2 1
2 0
2 1
2 0
2 

In [28]:
from sklearn.metrics import classification_report

# Assuming you have predicted labels (predicted_classes) and true labels (test_labels_encoded)
report = classification_report(labels, y_prob, output_dict=True)

# Access class-wise accuracy using 'accuracy' key in the report dictionary

# # Print accuracy for each class
# for class_name, accuracy in class_wise_accuracy.items():
#   print(f"Class {class_name}: Accuracy - {accuracy:.4f}")
for i in report:
  print(i, report[i])

0 {'precision': 0.16, 'recall': 0.1340782122905028, 'f1-score': 0.1458966565349544, 'support': 179}
1 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}
2 {'precision': 0.637478108581436, 'recall': 0.56875, 'f1-score': 0.6011560693641618, 'support': 640}
3 {'precision': 0.4398563734290844, 'recall': 0.546875, 'f1-score': 0.4875621890547264, 'support': 448}
accuracy 0.4949179046129789
macro avg {'precision': 0.30933362050263014, 'recall': 0.3124258030726257, 'f1-score': 0.30865372873846064, 'support': 1279}
weighted avg {'precision': 0.4954508559721258, 'recall': 0.4949179046129789, 'f1-score': 0.4920119207266128, 'support': 1279}


In [29]:
model.evaluate(images, labels)

40/40 [==============================] - 1s 22ms/step - loss: 2.8566 - sparse_categorical_accuracy: 0.4949 - f1_score: nan


[2.8566365242004395, 0.4949178993701935, nan]